# CRM Up-sell Prediction Project

For the second use case, up-sell / cross-sell, we will be using the same dataset by IBM, made available via Kaggle, already prepared and analysed in the `enriched_churn.ipynb` file. The up-sell / cross-sell use case may be integrated into the CRM functionality, as the ML functionality built in this project can be used to predict customers' potential to buy other adjacent products from the Telco firm.

The product to be up-sold is `device_protection_plan` — this is our target variable. In addition, we will remove `customer_status` — the target variable of our previous use-case, since the two are mutually exclusive (a customer that has churned cannot buy a new product).


## Project Description — Introduction:

### The Dataset:

The dataset is originally linked to IBM's Cognos Analytics 11.1.3+ Version Sample, made available via Kaggle. All data is property of IBM and/or its original author.

More information on the dataset can be found [here](https://www.kaggle.com/datasets/ylchang/telco-customer-churn-1113?select=Telco_customer_churn.xlsx).

The dataset being used has already been prepared in the first use case and 'dumped' into the `clean_data.csv` file.

### Technical Task:

We want to predict whether the product `device_protection_plan` can be up-sold / cross-sold to clients a USA Telco company based on client personal data concerning tariffs and contracts. If a custer is likely to accept an up-sell / cross-sell offer, the relationship manager should offer them that product.


### Data Description:

The dataset has been consolidated into a single file — `clean_data.csv`. The variable descriptions are given in the `enriched_churn.ipynb` file, where the data was pre-processed.


### Project Goal:

Based on the aforementioned data, we want to predict whether a customer will accept an up-sell / cross-sell product offer, which is a **binary classification** task.


### Specific Objectives, Metrics, and Implementation Plan:

**Goals:** since we already understand our data, we can simply pre-prepare our datasets, then build and test ML models that would enable us to predict the up-sell potential of the `device_protection_plan` product.

**Metrics:** we will be using the same F1 metric as in our previous use case, since our data is imbalanced (most people have not bought the product, which we know from the previous EDA).

**ML Plan:**
1. Analyse data for imbalance (done — significant imbalance detected in EDA).
2. Choose the best metric based on class imbalance (done — F1 metric selected).
3. Select models: we are dealing with a classification problem, so we will test several models. In this use case, our selection will not be as broad as before and we will be using:
    - Logistic regression
    - Random forest
    - Catboost classifier
4. Prepare the datasets (train, test, and validation) for both non-boosting and boosting models, including scaling and OHE.
5. Train and validate our models for preliminary results.
6. Pre-select best models.
6. Tune hyperparameters using cross-validation, validate best parameters on the validation dataset.
7. Select best model.
8. Apply to train dataset and analyse performance — compare it with a dummy model.
9. Draw conclusions.

<br>

## Dataset Preparation for Boosting and Non-boosting Models

In [1]:
# Importing all libraries necessary for this use-case:

import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import f1_score

from catboost import CatBoostClassifier

import time

In [2]:
cleaned_data = pd.read_csv('clean_data.csv', index_col='Unnamed: 0')
display(cleaned_data.head(), cleaned_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6536 entries, 0 to 6535
Data columns (total 34 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   satisfaction_score                 6536 non-null   int64  
 1   customer_status                    6536 non-null   int64  
 2   cltv                               6536 non-null   int64  
 3   number_of_referrals                6536 non-null   int64  
 4   tenure_in_months                   6536 non-null   int64  
 5   offer                              6536 non-null   object 
 6   phone_service                      6536 non-null   bool   
 7   avg_monthly_long_distance_charges  6536 non-null   float64
 8   multiple_lines                     6536 non-null   bool   
 9   internet_type                      6536 non-null   object 
 10  avg_monthly_gb_download            6536 non-null   int64  
 11  online_security                    6536 non-null   bool 

,satisfaction_score,customer_status,cltv,number_of_referrals,tenure_in_months,offer,phone_service,avg_monthly_long_distance_charges,multiple_lines,internet_type,...,total_refunds,total_extra_data_charges,total_long_distance_charges,population,latitude,longitude,gender,age,married,number_of_dependents
0,3,1,5433,0,1,None,False,0.00,False,DSL,...,0.00,20,0.00,68701,34.023810,-118.156582,Male,78,False,0
1,3,1,5302,1,8,Offer E,True,48.85,True,Fiber Optic,...,0.00,0,390.80,55668,34.044271,-118.185237,Female,74,True,1
2,2,1,3179,0,18,Offer D,True,11.33,True,Fiber Optic,...,45.61,0,203.94,47534,34.108833,-118.229715,Male,71,False,3
3,2,1,5337,1,25,Offer C,True,19.76,False,Fiber Optic,...,13.43,0,494.00,27778,33.936291,-118.332639,Female,78,True,1
4,2,1,2793,1,37,Offer C,True,6.33,True,Fiber Optic,...,0.00,0,234.21,26265,33.972119,-118.020188,Female,80,True,1


None

We must now remove the `customer_status` column, which is extraneous in this use-case.

In [3]:
cleaned_data = cleaned_data.drop('customer_status', axis=1)
cleaned_data.columns

Index(['satisfaction_score', 'cltv', 'number_of_referrals', 'tenure_in_months',
       'offer', 'phone_service', 'avg_monthly_long_distance_charges',
       'multiple_lines', 'internet_type', 'avg_monthly_gb_download',
       'online_security', 'online_backup', 'device_protection_plan',
       'premium_tech_support', 'streaming_tv', 'streaming_movies',
       'streaming_music', 'unlimited_data', 'contract', 'paperless_billing',
       'payment_method', 'monthly_charge', 'total_charges', 'total_refunds',
       'total_extra_data_charges', 'total_long_distance_charges', 'population',
       'latitude', 'longitude', 'gender', 'age', 'married',
       'number_of_dependents'],
      dtype='object')

Now we can pre-process our data along the same lines as in our previous use-case.

### Data Prepation for Non-boosting Models:

In [4]:
# Applying OHE, taking care to avoid extraneous dummy-variables:
df_nb = pd.get_dummies(cleaned_data.drop('device_protection_plan', axis=1), drop_first=True)

# nb = no boosting!
features_nb = df_nb
target_nb = cleaned_data['device_protection_plan']


# Function for scaling features:
def scaling(numeric, features_train, features_valid, features_test):
    scaler = StandardScaler()
    # Note: we are careful to teach the `Scaler` only on the training dataset:
    scaler.fit(features_train[numeric])
    features_train[numeric] = scaler.transform(features_train[numeric])
    features_valid[numeric] = scaler.transform(features_valid[numeric])
    features_test[numeric] = scaler.transform(features_test[numeric])
    return features_train, features_valid, features_test


# Function for dataset preparation:
def dataset_prep(features, target, numeric):
    features_train, features_test_and_valid, target_train, target_test_and_valid = train_test_split(features, target,
                                                                                                    test_size=0.3,
                                                                                                    random_state=22)
    features_valid, features_test, target_valid, target_test = train_test_split(features_test_and_valid,
                                                                                target_test_and_valid, test_size=0.5,
                                                                                random_state=22)
    # Data randomisation:
    features_train, target_train = shuffle(features_train, target_train, random_state=22)
    features_valid, target_valid = shuffle(features_valid, target_valid, random_state=22)
    features_test, target_test = shuffle(features_test, target_test, random_state=22)
    # Note: we could have also added shuffle=True to train_test_split, since with random_state the results would have been equal
    # Scaling numeric features:
    features_train, features_valid, features_test = scaling(numeric, features_train,
                                                            features_valid, features_test)
    return features_train, features_valid, features_test, target_train, target_valid, target_test


numeric = ['satisfaction_score', 'cltv', 'number_of_referrals', 'tenure_in_months',
           'avg_monthly_long_distance_charges', 'avg_monthly_gb_download', 'monthly_charge',
           'total_charges', 'total_refunds', 'total_extra_data_charges', 'total_long_distance_charges',
           'population', 'latitude', 'longitude', 'age', 'number_of_dependents']

features_train_nb, features_valid_nb, features_test_nb, target_train_nb, target_valid_nb, target_test_nb = dataset_prep(
    features_nb, target_nb, numeric)

display(features_train_nb.shape, features_valid_nb.shape, features_test_nb.shape)

(4575, 40)

(980, 40)

(981, 40)

### Data Preparation for Boosting models

For boosting-models, we can use categorical features without encoding:

- For Catboost: we need to specify categorical features in the `cat_features` parameter

In [5]:
cat_features  = ['offer', 'internet_type', 'contract', 'payment_method', 'gender']

df_cat = cleaned_data.drop('device_protection_plan', axis=1)

features_cat = df_cat
target_cat = cleaned_data['device_protection_plan']

features_train_cat, features_valid_cat, features_test_cat, target_train_cat, target_valid_cat, target_test_cat =\
    dataset_prep(features_cat, target_cat, numeric)

display(features_train_cat.shape, features_valid_cat.shape, features_test_cat.shape)

(4575, 32)

(980, 32)

(981, 32)

Now that we have our datasets prepared, we can move on to training baseline models.

## Baseline Model Training

Let us first initiate a dataframe with our results:

In [6]:
df_results = pd.DataFrame(index = ['f1_score', 'learning_time', 'prediction_time'])
df_results

""
f1_score
learning_time
prediction_time


For our CatBoost model, we will also need to calculate our class-imbalance ratio:

In [7]:
count = pd.Series(target_cat.value_counts())

scale_pos_weight = count[0] / count[1]
scale_pos_weight_rounded = scale_pos_weight.round()
display(scale_pos_weight_rounded)

2.0

We can now run our baseline models and compare their speed and F1 scores. Notably, we will be adjusting all our models for class imbalance:

In [8]:
pd.set_option('display.float_format', '{:.5f}'.format)

def modelling(model, features_train, target_train, features_valid, target_valid):
    start_time = time.time()
    model.fit(features_train, target_train)
    learning_time = time.time() - start_time
    predictions = model.predict(features_valid)
    prediction_time = time.time() - learning_time - start_time
    predictions = pd.Series(predictions).astype(bool)
    f1 = f1_score(target_valid, predictions)
    return [f1, learning_time, prediction_time]

# We are using practically default parameters in our models:
df_results = df_results.assign(LogisticRegression = modelling(LogisticRegression(random_state = 22, class_weight='balanced'), features_train_nb, target_train_nb, features_valid_nb, target_valid_nb))

df_results = df_results.assign(RandomForestClassifier = modelling(RandomForestClassifier(random_state = 22, class_weight='balanced'), features_train_nb, target_train_nb, features_valid_nb, target_valid_nb))

df_results = df_results.assign(CatboostClassifier = modelling(CatBoostClassifier(random_state = 22, scale_pos_weight = scale_pos_weight_rounded, cat_features=cat_features, silent=True, allow_writing_files=False), features_train_cat, target_train_cat, features_valid_cat, target_valid_cat))

display(df_results)

/Users/ana/opt/miniconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,LogisticRegression,RandomForestClassifier,CatboostClassifier
f1_score,0.93154,0.66252,0.53622
learning_time,0.09249,1.14197,6.75689
prediction_time,0.00481,0.02743,0.01262


Our best model is immediately proven to be `LogisticRegression`. It is also by far the fastest model. We could potentially engage in hyperparameter tuning in order to improve the performance of the other models; however, their current F1 score is significantly lower, while their learning time is higher. The difference in performance is so noticeable that it is unlikely that we will be able to increase the F1 score in order to match the outcome of `LogisticRegression`.

Since the `LogisticRegression` method does not include many hyperparameter to tune, we can skip that aspect of our ML analytics. Based on this reasoning, we have now selected the `LogisticRegression` model as our final one.

Therefore, we can now test our model on the testing dataset and compare its performance with a `DummyClassifier`.
<br>
### Test-dataset Performance and Dummy Comparison

In [9]:
df_results_test = pd.DataFrame(index = ['f1_score', 'learning_time', 'prediction_time'])

df_results_test = df_results_test.assign(LogisticRegression_final = modelling(LogisticRegression(random_state = 22, class_weight='balanced'), features_train_nb, target_train_nb, features_test_nb, target_test_nb))

df_results_test = df_results_test.assign(DummyClassifier = modelling(DummyClassifier(random_state=22, strategy='uniform'), features_train_nb, target_train_nb, features_test_nb, target_test_nb))

df_results_test

/Users/ana/opt/miniconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,LogisticRegression_final,DummyClassifier
f1_score,0.94180,0.43137
learning_time,0.14527,0.00111
prediction_time,0.00586,0.00088


Since our `LogisticRegression` model performs far better that the `DummyClassifier`, it has passed the necessary sanity check. We can now select our model for further application without our CRM.
<br>
## Project Outcomes Discussion

We have now completed the second use-case for our CRM project. We have already discussed the drawbacks of our dataset in the previous use-case; here, we can mention that we could apply a wider model selection to our data in order to verify that `LogisticRegression` is indeed the most suitable model for this use-case with the best and quickest results.

Having selected our model, we can now pre-train once more and then save both the model and our `StandardScaler` in order to implement these in our CRM web-app.

In [10]:
import joblib # Importing joblib for dumping our scaler

In [12]:
df_nb = pd.get_dummies(cleaned_data.drop('device_protection_plan', axis=1), drop_first=True)

features_nb = df_nb
target_nb = cleaned_data['device_protection_plan']

features_train, features_test_and_valid, target_train, target_test_and_valid = train_test_split(features_nb, target_nb,
                                                                                                    test_size=0.3,
                                                                                                    random_state=22)

features_valid, features_test, target_valid, target_test = train_test_split(features_test_and_valid,
                                                                                target_test_and_valid, test_size=0.5,
                                                                                random_state=22)

features_train, target_train = shuffle(features_train, target_train, random_state=22)

numeric = ['satisfaction_score', 'cltv', 'number_of_referrals', 'tenure_in_months',
           'avg_monthly_long_distance_charges', 'avg_monthly_gb_download', 'monthly_charge',
           'total_charges', 'total_refunds', 'total_extra_data_charges', 'total_long_distance_charges',
           'population', 'latitude', 'longitude', 'age', 'number_of_dependents']

scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])

model = LogisticRegression(random_state = 22, class_weight='balanced')

model.fit(features_train, target_train)

joblib.dump(scaler, 'upsell_scaler.bin', compress=True)

filename = 'upsell_model.sav'
joblib.dump(model, filename)

print("Model trained and saved successfully.")

Model trained and saved successfully.


/Users/ana/opt/miniconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [13]:
display(features_train.info(), features_train.columns, features_train)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4575 entries, 2320 to 2173
Data columns (total 40 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   satisfaction_score                 4575 non-null   float64
 1   cltv                               4575 non-null   float64
 2   number_of_referrals                4575 non-null   float64
 3   tenure_in_months                   4575 non-null   float64
 4   phone_service                      4575 non-null   bool   
 5   avg_monthly_long_distance_charges  4575 non-null   float64
 6   multiple_lines                     4575 non-null   bool   
 7   avg_monthly_gb_download            4575 non-null   float64
 8   online_security                    4575 non-null   bool   
 9   online_backup                      4575 non-null   bool   
 10  premium_tech_support               4575 non-null   bool   
 11  streaming_tv                       4575 non-null   bo

None

Index(['satisfaction_score', 'cltv', 'number_of_referrals', 'tenure_in_months',
       'phone_service', 'avg_monthly_long_distance_charges', 'multiple_lines',
       'avg_monthly_gb_download', 'online_security', 'online_backup',
       'premium_tech_support', 'streaming_tv', 'streaming_movies',
       'streaming_music', 'unlimited_data', 'paperless_billing',
       'monthly_charge', 'total_charges', 'total_refunds',
       'total_extra_data_charges', 'total_long_distance_charges', 'population',
       'latitude', 'longitude', 'age', 'married', 'number_of_dependents',
       'offer_Offer A', 'offer_Offer B', 'offer_Offer C', 'offer_Offer D',
       'offer_Offer E', 'internet_type_DSL', 'internet_type_Fiber Optic',
       'internet_type_None', 'contract_One Year', 'contract_Two Year',
       'payment_method_Credit Card', 'payment_method_Mailed Check',
       'gender_Male'],
      dtype='object')

,satisfaction_score,cltv,number_of_referrals,tenure_in_months,phone_service,avg_monthly_long_distance_charges,multiple_lines,avg_monthly_gb_download,online_security,online_backup,...,offer_Offer D,offer_Offer E,internet_type_DSL,internet_type_Fiber Optic,internet_type_None,contract_One Year,contract_Two Year,payment_method_Credit Card,payment_method_Mailed Check,gender_Male
2320,-1.81410,-0.01287,-0.66779,-1.31684,True,0.18871,False,-0.25060,True,False,...,0,1,0,0,0,0,0,0,1,0
4067,-0.16835,0.89177,-0.00672,1.56026,True,0.69627,True,-1.03111,False,False,...,0,0,0,0,1,0,1,1,0,1
369,-1.81410,-1.15750,-0.66779,-1.40024,True,0.45860,False,0.43235,False,False,...,0,1,0,1,0,0,0,0,0,0
5263,-0.16835,-0.64689,-0.66779,-0.44121,True,0.80191,False,-0.88476,True,False,...,0,0,0,1,0,0,0,0,0,0
2588,-0.16835,0.18287,0.32382,1.51856,True,0.58098,True,-1.03111,False,False,...,0,0,0,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5875,-0.16835,-0.00010,0.98489,1.56026,True,0.83347,True,-0.34816,True,False,...,0,0,0,1,0,0,1,0,0,1
2333,-0.16835,-0.45796,-0.33725,-1.23345,True,-0.52884,True,-0.83598,False,False,...,0,0,1,0,0,0,0,0,0,0
1265,-0.16835,0.79986,-0.66779,-1.40024,True,1.45247,False,0.33479,False,False,...,0,1,0,1,0,0,0,0,0,1
1773,-1.81410,-0.26563,-0.33725,-0.14933,True,-1.35202,True,-0.20181,False,True,...,0,0,0,1,0,0,0,0,0,1
